In [1]:
import pyspark  

sc = pyspark.SparkContext('local[*]') 

# do something to prove it works 

rdd = sc.parallelize(range(1000)) 

rdd.takeSample(False, 5) 

[713, 207, 522, 383, 266]

In [2]:
!ls

account.csv		  notebook.ipynb  transactions.csv
country_abbreviation.csv  parse_file.txt


In [6]:
account_df = spark.read.csv("../../data/accounts.csv", header=True)
account_count = account_df.count()
print(f"The account.csv file has {account_count} rows.")

The account.csv file has 500000 rows.


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CSV File Count Example") \
    .getOrCreate()

In [15]:
# Read account.csv file
account_df = spark.read.csv("../../data/accounts.csv", header=True,sep=";")
account_count = account_df.count()
print(f"The account.csv file has {account_count} rows.")

# Read transactions.csv file
transactions_df = spark.read.csv("../../data/transactions.csv", header=True,sep=";")
transactions_count = transactions_df.count()
print(f"The transactions.csv file has {transactions_count} rows.")

# Read country_abbreviation.csv file
country_abbreviation_df = spark.read.csv("../../data/country_abbreviation.csv", header=True,sep=";")
country_abbreviation_count = country_abbreviation_df.count()
print(f"The country_abbreviation.csv file has {country_abbreviation_count} rows.")

The account.csv file has 500000 rows.
The transactions.csv file has 5000000 rows.
The country_abbreviation.csv file has 121 rows.


In [16]:
transactions_df.show(5)

+------+-------+------------+----------------+-------+
|    id| amount|account_type|transaction_date|country|
+------+-------+------------+----------------+-------+
|179528|-730.86|    Business|      2013-07-10|     SV|
|378343|-946.98|    Personal|      2018-04-06|     YE|
| 75450|7816.92|Professional|      2016-11-20|     SI|
|357719| 704.02|    Business|      2016-11-06|     ID|
|110511| 3462.6|    Personal|      2018-01-18|     BS|
+------+-------+------------+----------------+-------+
only showing top 5 rows



In [21]:

# Spark SQL 2
transactions_df = spark.read.csv("../../data/transactions.csv", header=True,sep=";")

# Register the DataFrame as a SQL temporary table
transactions_df.createOrReplaceTempView("transactions")

# SQL Query to count the number of accounts of each type
account_type_count_df = spark.sql(
    "SELECT account_type, COUNT(DISTINCT id) as account_type_count "
    "FROM transactions "
    "GROUP BY account_type"
)

# Show the result
account_type_count_df.show()

+------------+------------------+
|account_type|account_type_count|
+------------+------------------+
|    Personal|            481997|
|Professional|            482170|
|    Business|            482350|
+------------+------------------+



In [24]:
from pyspark.sql import functions as F


# Convert amount to float and transaction_date to date type
transactions_df = transactions_df.withColumn("amount", F.col("amount").cast("float"))
transactions_df = transactions_df.withColumn("transaction_date", F.col("transaction_date").cast("date"))

# Register the DataFrame as a SQL temporary view
transactions_df.createOrReplaceTempView("transactions")

# SQL Query to calculate the balance and latest transaction date for each account
result_df = spark.sql(
    """
    SELECT 
        id, 
        SUM(amount) as balance, 
        MAX(transaction_date) as latest_date
    FROM transactions
    GROUP BY id
    """
)

# Convert balance to string type as per requirement
result_df = result_df.withColumn("balance", F.col("balance").cast("string"))

# Show the result
result_df.show()

+------+------------------+-----------+
|    id|           balance|latest_date|
+------+------------------+-----------+
| 68325|53232.409576416016| 2021-12-02|
|249223|30680.630126953125| 2020-04-13|
|113602|     54096.0703125| 2021-12-27|
|448517|52082.130615234375| 2021-02-04|
|193630| 63034.56997680664| 2021-02-05|
|417334|21550.009552001953| 2020-11-30|
|351084|  35345.1498708725| 2021-10-21|
|429742| 51014.89072418213| 2021-12-26|
|195983|18586.459869384766| 2021-08-27|
|454795| 68893.17016601562| 2021-02-21|
|444212| 40396.46002960205| 2020-09-08|
|414329| 45952.11022949219| 2019-04-26|
|191501| 73625.31985473633| 2021-10-09|
|376767|44612.060302734375| 2021-08-12|
|355412|29777.969772338867| 2021-07-04|
| 55456|48036.110580444336| 2019-10-15|
|383353|43718.770080566406| 2021-10-25|
|224178| 69300.92985534668| 2021-10-10|
|462160| 37512.43994140625| 2021-02-15|
|191369| 38177.33986663818| 2021-02-26|
+------+------------------+-----------+
only showing top 20 rows



In [35]:
account_df = spark.read.csv("../../data/accounts.csv", header=True,sep=";")

In [36]:
account_df.show(5)

+---+----------+---------+---+-------+
| id|first_name|last_name|age|country|
+---+----------+---------+---+-------+
|  1|     Darcy| Phillips| 24|     YE|
|  2|    Amelia|   Wright| 66|     CN|
|  3|     Haris|    Ellis| 61|     CR|
|  4|      Tony|     Hall| 51|     JO|
|  5|     Rubie|  Stewart| 27|     RO|
+---+----------+---------+---+-------+
only showing top 5 rows



In [44]:

# Convert amount to float and transaction_date to date type
transactions_df = transactions_df.withColumn("amount", F.col("amount").cast("float"))
transactions_df = transactions_df.withColumn("transaction_date", F.col("transaction_date").cast("date"))

# Extract year from transaction_date
transactions_df = transactions_df.withColumn("year", F.year("transaction_date"))

# Filter for Swiss users and join with transactions
swiss_users_df = account_df.filter(account_df.country == "CH") # Assuming "CH" stands for Switzerland
joined_df = transactions_df.join(swiss_users_df, transactions_df.id == swiss_users_df.id)

# Calculate user full name and filter for positive transactions (earnings)
joined_df = joined_df.withColumn("full_name", F.concat_ws(" ", "first_name", "last_name"))
positive_transactions_df = joined_df.filter(joined_df.amount > 0)

# Calculate total earnings for each Swiss user by year
pivot_df = positive_transactions_df.groupBy("full_name").pivot("year").sum("amount")

# Show the result
pivot_df.show(5)

+---------------+------------------+-----------------+----------------+-----------------+------------------+------------------+----------------+-----------------+-----------------+------------------+--------------+
|      full_name|              2011|             2012|            2013|             2014|              2015|              2016|            2017|             2018|             2019|              2020|          2021|
+---------------+------------------+-----------------+----------------+-----------------+------------------+------------------+----------------+-----------------+-----------------+------------------+--------------+
|  Connie Gibson|              null|    8486.83984375|3664.68994140625| 7385.68994140625|              null|  1717.47998046875|2436.81005859375|             null|   5545.759765625|              null|          null|
|  Justin Cooper|20224.619873046875| 5977.14990234375|            null|16800.74951171875|  6475.06982421875|              null|            n

In [45]:
from pyspark.sql.functions import col, expr

def add_transaction_level(spark, transactions_df):
    # Convert amount to float if it's not already
    transactions_df = transactions_df.withColumn("amount", col("amount").cast("float"))

    # Use SQL expressions to calculate quantiles directly
    transactions_with_level = transactions_df.withColumn(
        "level",
        expr(
            "CASE WHEN amount > percentile_approx(amount, 0.75) OVER () THEN 'high' "
            "WHEN amount > percentile_approx(amount, 0.25) OVER () AND amount <= percentile_approx(amount, 0.75) OVER () THEN 'average' "
            "ELSE 'low' END"
        )
    )

    # Return the DataFrame with the additional "level" column
    return transactions_with_level


In [46]:
transactions_with_level = add_transaction_level(spark, transactions_df)

In [47]:
transactions_with_level.show()

+------+-------+------------+----------------+-------+----+-------+
|    id| amount|account_type|transaction_date|country|year|  level|
+------+-------+------------+----------------+-------+----+-------+
|179528|-730.86|    Business|      2013-07-10|     SV|2013|    low|
|378343|-946.98|    Personal|      2018-04-06|     YE|2018|    low|
| 75450|7816.92|Professional|      2016-11-20|     SI|2016|   high|
|357719| 704.02|    Business|      2016-11-06|     ID|2016|    low|
|110511| 3462.6|    Personal|      2018-01-18|     BS|2018|average|
|461830| 762.81|Professional|      2017-06-20|     CN|2017|    low|
| 30180|5390.24|Professional|      2021-05-26|     GN|2021|average|
| 65398|4765.77|    Personal|      2018-05-01|     TR|2018|average|
|170899|8775.89|    Business|      2013-10-16|     SK|2013|   high|
|234300|8455.18|Professional|      2015-10-06|     LU|2015|   high|
|208027| 6244.1|    Business|      2020-03-06|     AE|2020|average|
|161212|5904.56|    Personal|      2016-09-07|  

In [49]:
file_path = 'parse_file.txt'
df = read_pyspark_output_to_dataframe(spark, file_path)
df.show(5)

+---+----+----+
| id|Col1|Col2|
+---+----+----+
|   |   i|   d|
|   |    |   1|
|   |    |   2|
|   |    |   3|
|   |    |   4|
+---+----+----+
only showing top 5 rows



In [48]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

def read_pyspark_output_to_dataframe(spark, file_path):
    # Read the text file as a DataFrame
    df = spark.read.text(file_path)

    # Define a regular expression pattern to match rows with alphanumeric characters
    pattern = r'[a-zA-Z0-9]+'

    # Filter the rows based on the pattern
    filtered_df = df.filter(col("value").rlike(pattern))

    # Extract columns from the first row
    columns = filtered_df.first().value.split("|")[1:-1]
    columns = [col.strip() for col in columns]

    # Split and clean the data for rows
    cleaned_df = filtered_df.selectExpr(
        *[f"split(value, '\\|')[{i+1}] as {col}" for i, col in enumerate(columns)]
    )

    # Return the DataFrame
    return cleaned_df



In [50]:
import pandas as pd
import re

def read_pyspark_output_to_dataframe(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Remove the separator lines and strip whitespace
    clean_lines = [line.strip() for line in lines if re.search(r'[a-zA-Z0-9]+', line)]
    columns_line = clean_lines[0]
    columns = columns_line.split("|")[1:-1]
    columns = [col.strip() for col in columns]

    # print(columns)
    rows = []
    for line in clean_lines[1:]:
        row_data = line.split("|")[1:-1]
        row_data = [item.strip() for item in row_data]
        rows.append(row_data)
    # print(rows)
    df = pd.DataFrame(rows, columns=columns)
    return df

# Example usage
file_path = 'parse_file.txt'
df = read_pyspark_output_to_dataframe(file_path)
print(df)


  id                Col1   Col2
0  1       one,two,three    one
1  2       four,one,five    six
2  3  seven,nine,one,two  eight
3  4      two,three,five   five
4  5        six,five,one  seven


In [55]:
def read_pyspark_output_to_dataframe(spark, file_path):
    # Read the text file as a DataFrame
    df = spark.read.text(file_path)

    # Define a regular expression pattern to match rows with alphanumeric characters
    pattern = r'[a-zA-Z0-9]+'

    # Filter the rows based on the pattern and remove separator lines
    filtered_df = df.filter(col("value").rlike(pattern) & (col("value").like("%|%")))

    # Extract columns from the first row
    columns_line = filtered_df.first().value
    columns = columns_line.split("|")[1:-1]
    columns = [col.strip() for col in columns]

    # Return the DataFrame
    return columns,filtered_df

file_path = 'parse_file.txt'
columns,df = read_pyspark_output_to_dataframe(spark, file_path)
df.show(5)



+--------------------+
|               value|
+--------------------+
|| id|            ...|
||  1|     one,two...|
||  2|     four,on...|
||  3|seven,nine,o...|
||  4|    two,thre...|
+--------------------+
only showing top 5 rows



In [64]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col

def read_pyspark_output_to_dataframe(spark, file_path):
    # Read the text file as a DataFrame
    df = spark.read.text(file_path)

    # Define a regular expression pattern to match rows with alphanumeric characters
    pattern = r'[a-zA-Z0-9]+'

    # Filter the rows based on the pattern and remove separator lines
    filtered_df = df.filter(col("value").rlike(pattern) & (col("value").like("%|%")))

    # Extract column names from the first row (skip the first line)
    first_row = filtered_df.first()
    columns = [col.strip() for col in first_row.value.split("|")[1:-1]]

    # Remove the first line (header)
    data_df = filtered_df.dropDuplicates(subset=["value"])

    # Split the remaining lines into columns based on "|"
    for i, col_name in enumerate(columns):
        data_df = data_df.withColumn(col_name, split(data_df["value"], "\\|")[i + 1])

    # Drop the original "value" column
    data_df = data_df.drop("value")

    # Return the DataFrame
    return data_df

# Create a Spark session


In [65]:
file_path = 'parse_file.txt'
df = read_pyspark_output_to_dataframe(spark, file_path)


In [66]:
df.show(5)

+---+------------------+-----+
| id|              Col1| Col2|
+---+------------------+-----+
| id|              Col1| Col2|
|  5|      six,five,one|seven|
|  2|     four,one,five|  six|
|  1|     one,two,three|  one|
|  3|seven,nine,one,two|eight|
+---+------------------+-----+
only showing top 5 rows

